<a href="https://colab.research.google.com/github/yuhrtm/Tibame_GAD245/blob/main/%E6%8A%93%E9%AB%98%E9%90%B5%E7%B6%B2%E7%AB%99%E6%99%82%E5%88%BB%E8%A1%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd
import logging
from tabulate import tabulate

# 設置 logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def fetch_high_speed_rail_schedule(start_station, end_station, outward_date, outward_time, return_date=None, return_time=None):
    """
    從台灣高鐵 API 獲取時刻表數據。
    """
    url = "https://www.thsrc.com.tw/TimeTable/Search"
    payload = {
        "SearchType": "S" if not return_date else "R",  # S: 單程, R: 來回
        "Lang": "TW",
        "StartStation": start_station,
        "EndStation": end_station,
        "OutWardSearchDate": outward_date,
        "OutWardSearchTime": outward_time,
        "ReturnSearchDate": return_date or "",
        "ReturnSearchTime": return_time or "",
        "DiscountType": ""
    }
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Content-Type": "application/json",
        "Referer": "https://www.thsrc.com.tw/"
    }

    try:
        logging.info("向高鐵 API 發送請求...")
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        data = response.json()
        logging.info("成功獲取高鐵時刻表數據。")
        return data
    except requests.exceptions.RequestException as e:
        logging.error(f"請求失敗: {e}")
        return None
    except json.JSONDecodeError:
        logging.error("回應數據非 JSON 格式，無法解析。")
        return None

def parse_schedule_data(data):
    """
    解析 JSON 數據為 Pandas DataFrame，加入出發站與抵達站資訊。
    """
    try:
        departure_table = data.get("data", {}).get("DepartureTable", {})
        train_items = departure_table.get("TrainItem", [])
        title = departure_table.get("Title", {})

        start_station_name = title.get("StartStationName", "無資料")
        end_station_name = title.get("EndStationName", "無資料")

        if not train_items:
            logging.warning("未找到任何時刻表數據。")
            return pd.DataFrame()  # 回傳空的 DataFrame

        # 將數據轉換為結構化表格
        parsed_data = [
            {
                "車次": trip.get("TrainNumber", "無資料"),
                "出發時間": trip.get("DepartureTime", "無資料"),
                "抵達時間": trip.get("DestinationTime", "無資料"),
                "行車時間": trip.get("Duration", "無資料"),
                "非對號車": trip.get("NonReservedCar", "無資料"),
                "行駛日期": trip.get("RunDate", "無資料"),
                "出發站": start_station_name,
                "抵達站": end_station_name
            }
            for trip in train_items
        ]

        return pd.DataFrame(parsed_data)
    except Exception as e:
        logging.error(f"解析數據失敗: {e}")
        return pd.DataFrame()

def display_schedule(schedule):
    """
    美化顯示時刻表。
    """
    if schedule.empty:
        logging.warning("數據為空，無法顯示。")
        return

    # 使用 Tabulate 格式化輸出
    print("\n高鐵時刻表如下：\n")
    print(tabulate(schedule, headers="keys", tablefmt="grid", showindex=False))
    print("\n提示：以上數據僅供參考，實際車次以台灣高鐵官方為準。\n")

def save_schedule_to_json(schedule, filename="high_speed_rail_schedule.json"):
    """
    將 Pandas DataFrame 保存為 JSON 文件。
    """
    if schedule.empty:
        logging.warning("數據為空，無法保存。")
        return

    try:
        schedule.to_json(filename, orient="records", force_ascii=False, indent=4)
        logging.info(f"時刻表已保存至 {filename}")
    except Exception as e:
        logging.error(f"保存數據失敗: {e}")

def main():
    """
    主程式：查詢高鐵時刻表並保存結果。
    """
    start_station = "NanGang"
    end_station = "ZuoYing"
    outward_date = "2025/01/20"
    outward_time = "11:00"

    logging.info("正在查詢高鐵時刻表，請稍候...")
    data = fetch_high_speed_rail_schedule(start_station, end_station, outward_date, outward_time)

    if data:
        schedule = parse_schedule_data(data)
        if not schedule.empty:
            display_schedule(schedule)  # 美化顯示
            save_schedule_to_json(schedule)  # 保存結果
        else:
            logging.warning("無法解析數據或數據為空。")
    else:
        logging.error("查詢失敗，請檢查網路或參數設置。")

if __name__ == "__main__":
    main()



高鐵時刻表如下：

+--------+------------+------------+------------+------------+------------+----------+----------+
|   車次 | 出發時間   | 抵達時間   | 行車時間   | 非對號車   | 行駛日期   | 出發站   | 抵達站   |
+========+============+============+============+============+============+==========+==========+
|   0803 | 06:15      | 08:40      | 02:25      | 9-12       | 2025/01/20 | 南港     | 左營     |
+--------+------------+------------+------------+------------+------------+----------+----------+
|   1103 | 06:35      | 08:20      | 01:45      | 10-12      | 2025/01/20 | 南港     | 左營     |
+--------+------------+------------+------------+------------+------------+----------+----------+
|   0603 | 06:40      | 08:50      | 02:10      | 9-12       | 2025/01/20 | 南港     | 左營     |
+--------+------------+------------+------------+------------+------------+----------+----------+
|   0805 | 07:00      | 09:25      | 02:25      | 9-12       | 2025/01/20 | 南港     | 左營     |
+--------+------------+------------+------------+----